<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Batch Ingestion for Binary Classification (Classification, AUC, Log Loss and Regression Metrics)

This example walks through the Arize `pandas` batch SDK for [ingesting binary classification data with support for classification, AUC, log loss and regression metrics](https://docs.arize.com/arize/sending-data-to-arize/model-types/binary-classification/binary-classification-complete). Guides for other model types are available [here](https://docs.arize.com/arize/sending-data-to-arize/model-types).

## Install and Import Dependencies

In [ ]:
!pip install -q arize

import datetime

from arize.pandas.logger import Client
from arize.utils.types import ModelTypes, Environments, Schema
import numpy as np
import pandas as pd

## Download and Display Data

Note the presence of predicted labels, predicted scores, actual labels and actual scores (in this case, actual scores assume binary values, 1 for the positive class and 0 for the negative class). All four are required to compute the full suite of classification, AUC, log loss and regression metrics.

In [ ]:
df = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/documentation-sample-data/data-ingestion/binary-classification-assets/binary_classification_data.csv?raw=true",
    index_col=False,
)
feature_column_names = df.columns[1:-4]
df.head()

## Add Timestamps for Predictions

Generate sample timestamps for each prediction. More information on timestamps in Arize can be found [here](https://docs.arize.com/arize/sending-data-to-arize/model-schema-reference/timestamp).

In [ ]:
current_time = datetime.datetime.now().timestamp()

earlier_time = (
    datetime.datetime.now() - datetime.timedelta(days=30)
).timestamp()

optional_prediction_timestamps = np.linspace(
    earlier_time, current_time, num=df.shape[0]
)

df.insert(1, "prediction_ts", optional_prediction_timestamps.astype(int))
df[["prediction_ts"]].head()

## Create Arize Client

Sign up/ log in to your Arize account [here](https://app.arize.com/auth/login). Find your [space and API keys](https://docs.arize.com/arize/product-guides/sending-data). Copy/paste into the cell below.

In [ ]:
SPACE_KEY = "SPACE_KEY"  # Change this line.
API_KEY = "API_KEY"  # Change this line.
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)
if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ CHANGE SPACE AND API KEYS")
else:
    print("✅ Arize client setup done! Now you can start using Arize!")

## Define Schema

Create your [model schema](https://docs.arize.com/arize/sending-data-to-arize/model-schema-reference).

In [ ]:
schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="predicted_label",
    prediction_score_column_name="predicted_score",
    actual_label_column_name="actual_label",
    actual_score_column_name="actual_score",
    feature_column_names=feature_column_names,
)

## Log Data to Arize

Log the DataFrame using the [pandas API](https://docs.arize.com/arize/sending-data-to-arize/data-ingestion-methods/sdk-reference/python-sdk/arize.pandas).

In [ ]:
response = arize_client.log(
    dataframe=df,
    schema=schema,
    model_id="binary-classification-auc-log-loss-regression-batch-ingestion-tutorial",
    model_version="1.0.0",
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.PRODUCTION,
)

if response.status_code == 200:
    print(f"✅ Successfully logged data to Arize!")
else:
    print(
        f'❌ Logging failed with status code {response.status_code} and message "{response.text}"'
    )